In [1]:
import tensorflow as tf
import numpy as np
import os
import pickle
from DDPG.DDPG import DDPG_agent
from env.EnvHeightControl import EnvHeightControl

L:\Anaconda3\envs\PT\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
tf.set_random_seed(22)
np.set_printoptions(precision=3, suppress=True)

In [3]:
config = tf.ConfigProto(allow_soft_placement=True)
config.gpu_options.allow_growth = True
sess = tf.Session(config=config)


In [4]:

env = EnvHeightControl()
state_shape = 1
action_bound = 1
action_dim = 1
agent = DDPG_agent(sess, state_shape, action_bound, action_dim,minibatch_size = 32)
sess.run(tf.global_variables_initializer())
saver = tf.train.Saver(max_to_keep = 100)


Waiting for connection: 

TransportError: Retry connection over the limit

In [ ]:
agent.load(saver,"temp/DDPG_s4/1000")
#init = tf.variables_initializer(var_list, name='init')  
#sess.run(init)
#from DDPG.Critic import Critic
#agent.critic = Critic(sess, agent.state_shape, agent.action_dim, agent.minibatch_size,lr = 0.001, tau=0.001)

In [8]:

episode_count, success, episode_reward, step_count = 0, 0, 0, 0
treward = []
state = env.reset()
tsucc = []
tstep = []
tres = []

In [9]:

while True:

    action = agent.act(state)
    next_state, reward, done, info = env.step(action)
    episode_reward += reward
    agent.observe(state, action, reward, next_state, done)
    agent.train(times = 1)
    state = next_state
    step_count += 1
    print ("action: {}".format(action).ljust(20," "),"abso height: {:.2f}".format(state[1][0]*50).ljust(20," "), "reward: {:.5f}.".format(reward).ljust(20," "),"steps: {}".format(step_count).ljust(20," "),end = "\r")

    if done:

        if info == "success":
            success += 1
        print (" "*80,end = "\r")
        print("episode {} finish, reward: {:.5f}, total success: {} result: {} step: {}".format(episode_count, episode_reward, success, info, step_count).ljust(80," "))
        treward.append(episode_reward)
        tsucc.append(success)
        tstep.append(step_count)
        tres.append(info)
        episode_reward = 0
        step_count = 0
        episode_count += 1
        if episode_count % 50 == 0:
            filename = "temp/DDPG_s5"
            agent.save(saver,filename + "/{}/".format(str(episode_count)))
            pickle.dump({"reward":treward,"succ":tsucc,"step":tstep,"info":tres},
                        open(filename+"/reward_list_{}.pickle".format(str(episode_count)),"wb"))
        #if episode_count >= 100:
        #    break
        state = env.reset()


episode 0 finish, reward: 8.25977, total success: 0 result: collision step: 21              
episode 1 finish, reward: 10.10694, total success: 0 result: collision step: 26            
episode 2 finish, reward: 5.28298, total success: 0 result: collision step: 13            
episode 3 finish, reward: 13.75483, total success: 0 result: collision step: 37             
episode 4 finish, reward: 7.82022, total success: 0 result: collision step: 20              
episode 5 finish, reward: 7.85780, total success: 0 result: collision step: 20             
episode 6 finish, reward: 10.17718, total success: 0 result: collision step: 27            
episode 7 finish, reward: 13.27451, total success: 0 result: collision step: 37            
episode 8 finish, reward: 5.84048, total success: 0 result: collision step: 14             
episode 9 finish, reward: 4.52948, total success: 0 result: collision step: 12            
episode 10 finish, reward: 5.64461, total success: 0 result: collision step: 17

KeyboardInterrupt: 

In [ ]:
#test
agent.load(saver,"temp/premodel2/")

#env = EnvHeightControl()
env = EnvHeightControl(start = [-19,0,-6],aim = [-19, -82, -6])
#env = EnvHeightControl(start = [-28,0,-10],aim = [-28, 125, -10])

In [50]:
episode_count, success, episode_reward, step_count = 0, 0, 0, 0
state = env.reset()
while True:

    action = agent.act(state)#,noise = False)
    next_state, reward, done, info = env.step(action)
    #print(reward)
    episode_reward += reward
    state = next_state
    step_count += 1
    print ("aim height: {}".format(env.aim_height).ljust(20," "),"abso height: {:.2f}".format(state[1][0]*50).ljust(20," "), "reward: {:.5f}.".format(reward).ljust(20," "),"steps: {}".format(step_count).ljust(20," "),end = "\r")

    if done:

        if info == "success":
            success += 1
        print (" "*80,end = "\r")
        print("episode {} finish, average reward: {:.5f}, total success: {} result: {} step: {}".format(episode_count, episode_reward, success, info, step_count).ljust(80," "))
        episode_reward = 0
        step_count = 0
        episode_count += 1
        state = env.reset()

episode 0 finish, average reward: 3.82005, total success: 0 result: collision step: 13
episode 1 finish, average reward: 4.51422, total success: 0 result: collision step: 15
episode 2 finish, average reward: 3.80979, total success: 0 result: collision step: 13
episode 3 finish, average reward: 3.76679, total success: 0 result: collision step: 13


KeyboardInterrupt: 

In [11]:
#baseline
env.reset()
rt = 0
st = 0
while True:
    if env.getPos()[2]>-20:
        a = 1
    elif env.getPos()[2]<-25:
        a = -1
    else:
        a = 0
    s,r,done,info = env.step(a)
    rt += r
    st += 1
    print (info,r)
    if done:
        print (rt)
        break
        
            

None 0.457184944152832
None 0.33835807800292966
None 0.507097396850586
None 0.4613670349121094
None 0.4142881393432617
None 0.368560791015625
None 0.3217136383056641
None 0.2750931549072266
None 0.22914833068847656
None 0.18230262756347657
None 0.15928512573242187
None 0.16163055419921876
None 0.16393821716308593
None 0.16642776489257813
None 0.16879226684570312
None 0.17115676879882813
None 0.17346443176269533
None 0.17588577270507813
None 0.1783753204345703
None 0.18078529357910156
None 0.18321800231933594
None 0.18558250427246092
None 0.18794700622558594
None 0.19031150817871093
None 0.1927442169189453
None 0.19516555786132814
None 0.19748458862304688
None 0.19983772277832032
None 0.175841064453125
None 0.1519257354736328
None 0.1543964385986328
None 0.15687461853027343
None 0.15929595947265626
None 0.16167182922363282
None 0.16409317016601563
None 0.16657135009765625
None 0.1689472198486328
None 0.1713799285888672
None 0.17385810852050781
None 0.17627944946289062
None 0.17876899719